In [1]:
# For following a second Drone
#Must have second drone in sights of Mambo Drone
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import random
import tensorflow as tf
from sklearn.utils import shuffle



In [2]:
import pickle
import zipfile 

In [3]:
training_file = "./traffic-signs-data/train.p"
validation_file= "./traffic-signs-data/valid.p"
testing_file = "./traffic-signs-data/valid.p"

with open(training_file, mode = 'rb') as f:
    train = pickle.load(f)

with open(testing_file, mode = 'rb') as f:
    test = pickle.load(f)
    
with open(validation_file, mode = 'rb') as f:
    valid = pickle.load(f)
    

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']
X_valid, y_valid = valid['features'], valid['features']


In [4]:
import numpy as np

n_train = len(X_train)
n_test = len(X_test)
n_valid = len(X_valid)

#Image shape and dimensions
image_shape = X_train[0].shape
#Number of classes and labels

n_classes = len(np.unique(y_train))

print("Number of training examples = ", n_train)
print("Number of testing examples = ", n_test)
print("Image shape = ", image_shape)
print("Number of classes = ", n_classes)

Number of training examples =  34799
Number of testing examples =  4410
Image shape =  (32, 32, 3)
Number of classes =  43


In [5]:
#Preprocess the Dataset 

#Grayscale 

X_train_gray = np.sum(X_train / 3, axis = 3, keepdims=True)
X_valid_gray = np.sum(X_valid / 3, axis = 3, keepdims=True)
X_test_gray = np.sum(X_test / 3, axis = 3, keepdims= True)
print("X_train_gray", X_train_gray.shape)

X_train_gray (34799, 32, 32, 1)


In [6]:
# Normalize images 

X_train_normalize = ((X_train_gray - 128) / 128)
X_test_normalize = ((X_test_gray - 128) / 128)
X_valid_normalize = ((X_valid_gray - 128) / 128)

print(np.mean(X_train_normalize))
X_valid = X_valid_normalize
X_train = X_train_normalize
X_test = X_test_normalize

-0.354081335648


In [7]:
#Image Augmentation Functions 
import skimage

In [8]:
def sharpen_image(img):
    gb = cv2.GaussianBlur(img, (5,5), 20.0)
    return cv2.addWeighted(img, 2, gb, -1, 0)

In [9]:
def translate_random(img):
    np.random.seed(43)
    rand_x = np.random.randint(-5, 5)
    rand_y = np.random.randint(-5, 5)
    translation_matrix = np.float32([ [1,0,rand_x], [0,1,rand_y]])
    return cv2.warpAffine(img, translation_matrix, (32, 32))

In [10]:
def add_noise(img):
    noisy_image = skimage.util.random_noise(img, mode='gaussian', seed=None, clip=True) 
    return np.asarray(noisy_image, dtype="uint8")

In [11]:
def augment_all(data):
    for x in data:
        rand_number = random.randint(0, 6)
        if rand_number == 0 or rand_number == 1:
            x = add_noise(x)
        if rand_number == 2:
            x = translate_random(x)
        if rand_number == 3:
            x = sharpen_image(x)
    return data

In [14]:
EPOCHS = 12
BATCH_SIZE = 120

In [44]:
from tensorflow.contrib.layers import flatten

def lanet(x):
    mu = 0
    sigma = 0.1
    strides = [1, 1, 1, 1]
    strides_for_pool = [1, 2, 2, 1]
    
    #Input: 32, 32, 1 Output: 30, 30, 10
    weights_0 = tf.Variable(tf.truncated_normal(shape = (3, 3, 1, 10), mean=mu, stddev=sigma))
    bias_0 = tf.Variable(tf.zeros(10))
    
    conv_0 = tf.nn.conv2d(x, weights_0, strides, padding="VALID") + bias_0
    
    #Activation
    conv_0 = tf.nn.relu(conv_0)
    conv_0 = tf.nn.dropout(conv_0, 0.9)
    
    #Input 30, 30, 10 Output: 28, 28, 100
    weights_1 = tf.Variable(tf.truncated_normal(shape = (3, 3, 10, 100), mean=mu, stddev=sigma))
    bias_1 = tf.Variable(tf.zeros(100))
    
    conv_1 = tf.nn.conv2d(conv_0, weights_1, strides, padding="VALID") + bias_1
    
    #Activation
    conv_0 = tf.nn.relu(conv_1)
    
    #Input 28, 28, 100 Output: 14, 14, 100
    filter_shape = [1, 2, 2, 1]
    pool_1 = tf.nn.max_pool(conv_1, filter_shape, strides_for_pool, padding= "VALID")
    
    #Input 14, 14, 100 Output 10, 10, 150
    weights_2 = tf.Variable(tf.truncated_normal(shape=(5, 5, 100, 150), mean = mu, stddev=sigma))
    bias_2 = tf.Variable(tf.zeros(150))
    
    conv_2 = tf.nn.conv2d(pool_1, weights_2, strides, padding= "VALID") + bias_2
    
    #Activation
    conv_0 = tf.nn.relu(conv_0)
    conv_0 = tf.nn.dropout(conv_0, 0.8)
    
    #Maxpool 10 10 150 output 5 5 150
    filter_shape_2 = [1, 2, 2, 1]
    pool_2 = tf.nn.max_pool(conv_2, filter_shape_2, strides_for_pool, padding="VALID")
    
    #Input 5, 5, 150 Output 4, 4, 250
    weights_3 = tf.Variable(tf.truncated_normal(shape=(2, 2, 150, 250), mean= mu, stddev=sigma))
    bias_3 = tf.Variable(tf.zeros(250))
    
    conv_3 = tf.nn.conv2d(pool_2, weights_3, strides, padding= 'VALID') + bias_3
    
    #Activation 
    conv_3 = tf.nn.relu(conv_3)
    conv_3 = tf.nn.dropout(conv_3, 0.7)
    
    #Maxpool 4, 4, 250 Output 2, 2, 250
    filter_shape_3 = [1, 2, 2, 1]
    pool_3 = tf.nn.max_pool(conv_3, filter_shape_3, strides_for_pool, padding='VALID')
    
    #Layer 4 Input 2 2 250 output 1 1 1000
    weights_4 = tf.Variable(tf.truncated_normal(shape= (2, 2, 250, 1000), mean = mu, stddev = sigma))
    bias_4 = tf.Variable(tf.zeros(1000))
    
    conv_4 = tf.nn.conv2d(pool_3, weights_4, strides, padding= "VALID") + bias_4
    
    #Activation
    conv_4 = tf.nn.relu(conv_4)
    conv_4 = tf.nn.dropout(conv_4, 0.6)
    
    

    #Flatten
    fc1 = tf.contrib.layers.flatten(conv_4)
    
    #Output 1000 to 500
    wd2 = tf.Variable(tf.truncated_normal((1000, 500), mean=mu, stddev=sigma))
    bd2 = tf.Variable(tf.zeros(500))
    
    fc2 = tf.add(tf.matmul(fc1, wd2), bd2)
    
    #Activate 
    fc2 = tf.nn.relu(fc2)
    
    
    #fc2 = tf.nn.dropout(fc2, dropout)
    
    #Flatten 500 to 300
    
    wd3 = tf.Variable(tf.truncated_normal((500, 300), mean=mu, stddev=sigma))
    bd3 = tf.Variable(tf.zeros(300))
    
    fc3 = tf.add(tf.matmul(fc2, wd3), bd3)
    
    #Activate 
    fc3 = tf.nn.relu(fc3)
    fc3 = tf.nn.dropout(fc3, 0.6)
    
    #Input 300 to 120
    
    wd4 = tf.Variable(tf.truncated_normal((300, 120), mean=mu, stddev= sigma))
    bd4 = tf.Variable(tf.zeros(120))
    
    fc4 = tf.add(tf.matmul(fc3, wd4), bd4)
    
    #Activate
    fc4 = tf.nn.relu(fc4)
    #fc4 = tf.nn.dropout(fc4, dropout)
    
    #Input 120 to 84
    
    wd5 = tf.Variable(tf.truncated_normal((120, 84), mean=mu, stddev = sigma))
    bd5 = tf.Variable(tf.zeros(84))
    
    fc5 = tf.add(tf.matmul(fc4, wd5), bd5)
    
    #Activate
    fc5 = tf.nn.relu(fc5)
    
    #Flatten 84 to n_classes
    
    wd6 = tf.Variable(tf.truncated_normal((84, n_classes), mean=mu, stddev=sigma))
    bd6 = tf.Variable(tf.zeros(n_classes))
    
    logits = tf.add(tf.matmul(fc5, wd6), bd6)
    
    #Activate 
    logits = tf.nn.relu(logits)
    #logits = tf.nn.dropout(logits, dropout)
    
    return logits

In [45]:
x = tf.placeholder(tf.float32,(None, 32, 32, 1))
y = tf.placeholder(tf.int32,(None))
#Giving values of either 1 or 0
one_hot_y = tf.one_hot(y, n_classes)


In [46]:
#Training Pipeline 
learning_rate = 0.003
logits = lanet(x)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

training_operation = optimizer.minimize(loss_operation)


In [47]:
#Model Visualization
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for i in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[i: i + BATCH_SIZE], y_data[i: i + BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict = {x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
#Running the Training model

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    print("Train")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        #Augment Functions for shuffled Images
        X_train = augment_all(X_train)
        total_accuracy = 0
        sess = tf.get_default_session()
        for j in range(0, num_examples, BATCH_SIZE):
            #Trained in Batch Sizes
            batch_x, batch_y = X_train[j : j + BATCH_SIZE], y_train[j: j + BATCH_SIZE]
            sess.run(training_operation, feed_dict = {x: batch_x, y: batch_y})
        
        validation_accuracy = evaluate(X_valid, y_valid)
        
        
        print("EPOCHS", (i + 1))
        print("Validation Accuracy ... ", validation_accuracy)
        
        print()
        
        saver.save(sess, './boom1')
    print("Model saved")
    